In [5]:
from typing import AsyncGenerator
from google.adk.agents import LoopAgent, LlmAgent, BaseAgent
from google.adk.events import Event, EventActions
from google.adk.agents.invocation_context import InvocationContext
from dotenv import load_dotenv
import os

# Load environment variables (GOOGLE_API_KEY)
load_dotenv()

# Verify API key is loaded
if not os.getenv("GOOGLE_API_KEY"):
    print("WARNING: GOOGLE_API_KEY not found in environment variables!")
    print("Please add GOOGLE_API_KEY to your .env file")

# Best Practice: Define custom agents as complete, self-describing classes.
class ConditionChecker(BaseAgent):
    """A custom agent that checks for a 'completed' status in the session state."""
    name: str = "ConditionChecker"
    description: str = "Checks if a process is complete and signals the loop to stop."

    async def _run_async_impl(
        self, context: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        """Checks state and yields an event to either continue or stop the loop."""
        status = context.session.state.get("status", "pending")
        is_done = (status == "completed")
        
        print(f"  [ConditionChecker] Status: {status}, is_done: {is_done}")

        if is_done:
            # Escalate to terminate the loop when the condition is met.
            content = types.Content(
                role='model',
                parts=[types.Part(text="Condition met, terminating loop.")]
            )
            yield Event(author=self.name, content=content, actions=EventActions(escalate=True))
        else:
            # Yield an event with proper Content object to continue the loop.
            content = types.Content(
                role='model',
                parts=[types.Part(text="Condition not met, continuing loop.")]
            )
            yield Event(author=self.name, content=content)

# Correction: The LlmAgent must have a model and clear instructions.
process_step = LlmAgent(
    name="ProcessingStep",
    model="gemini-2.0-flash-exp",
    instruction="You are a step in a longer process. Perform your task. If you are the final step, update session state by setting 'status' to 'completed'."
)

# The LoopAgent orchestrates the workflow.
poller = LoopAgent(
    name="StatusPoller",
    max_iterations=10,
    sub_agents=[
        process_step,
        ConditionChecker() # Instantiating the well-defined custom agent.
    ]
)

# This poller will now execute 'process_step' and then 'ConditionChecker'
# repeatedly until the status is 'completed' or 10 iterations have passed.

print("Poller agent defined successfully!")

Poller agent defined successfully!


In [6]:
# --- Execute the poller agent ---
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from dotenv import load_dotenv
import asyncio

# Load environment variables (GOOGLE_API_KEY)
load_dotenv()

# Define variables for session setup
APP_NAME = "status_poller_app"
USER_ID = "user_123"
SESSION_ID = "session_loop_001"

async def run_poller_agent(query: str):
    """
    Helper function to call the poller agent with a query.
    """
    # Session and Runner setup
    session_service = InMemorySessionService()
    session = await session_service.create_session(
        app_name=APP_NAME, 
        user_id=USER_ID, 
        session_id=SESSION_ID
    )
    
    # Initialize session state
    session.state["status"] = "pending"
    
    runner = Runner(
        agent=poller, 
        app_name=APP_NAME, 
        session_service=session_service
    )

    content = types.Content(role='user', parts=[types.Part(text=query)])
    
    print(f"\n--- Running Status Poller with query: '{query}' ---")
    print(f"Initial session state: {session.state}")
    print("\n")
    
    iteration_count = 0
    
    # Use async iteration for run_async
    async for event in runner.run_async(
        user_id=USER_ID, 
        session_id=SESSION_ID, 
        new_message=content
    ):
        # Print event information
        if hasattr(event, 'author'):
            print(f"[Iteration {iteration_count}] Event from: {event.author}")
        
        if hasattr(event, 'content') and event.content:
            if hasattr(event.content, 'text') and event.content.text:
                print(f"  Content: {event.content.text[:100]}...")
        
        if event.is_final_response() and event.content:
            iteration_count += 1
            final_response = ""
            
            # Extract text response
            if hasattr(event.content, 'text') and event.content.text:
                final_response = event.content.text
            elif event.content.parts:
                text_parts = [part.text for part in event.content.parts if part.text]
                final_response = "".join(text_parts)
            
            print("\n" + "=" * 80)
            print("Poller Agent Final Response:")
            print("=" * 80)
            print(final_response)
            print("=" * 80)
            
            # Check final session state
            final_session = await session_service.get_session(
                app_name=APP_NAME,
                user_id=USER_ID,
                session_id=SESSION_ID
            )
            print(f"\nFinal session state: {final_session.state}")
            print(f"Total iterations: {iteration_count}")

# Run the poller agent
await run_poller_agent("Start the processing loop and complete when ready")


--- Running Status Poller with query: 'Start the processing loop and complete when ready' ---
Initial session state: {'status': 'pending'}


[Iteration 0] Event from: ProcessingStep

Poller Agent Final Response:
Okay, I will start the processing loop and complete it when ready. I will now wait for further instructions or data to process. I will regularly check if the 'status' in the session state has been set to 'completed'. Once it is, I will cease processing.


Final session state: {}
Total iterations: 1
  [ConditionChecker] Status: pending, is_done: False
[Iteration 1] Event from: ConditionChecker

Poller Agent Final Response:
Condition not met, continuing loop.

Final session state: {}
Total iterations: 2
[Iteration 2] Event from: ProcessingStep

Poller Agent Final Response:
Okay, the condition check failed, so I will continue the processing loop and wait for the next instruction.


Final session state: {}
Total iterations: 3
  [ConditionChecker] Status: pending, is_done: False
[I